In [1]:
from submodules.Scenes import *
from submodules.Objects import *
from submodules.Actions import * 

# Object features
- Each object has functions, how we can interact with the object and change its inner logic variables.
- Each object instance has its own feasibility logic structure
- Object functions operates one level under `Actions` class. The users are triggering actions and do NOT use object functions

## Ex. 1: Put cup1 into drawer

In [2]:
drawer1 = Objects.Drawer(name='drawer1', position=[0,0,0])
drawer2 = Objects.Drawer(name='drawer2', position=[2,0,0])
cup1 = Objects.Cup(name='cup1', position=[2,0,0])

In [3]:
drawer2.open()

True

In [4]:
drawer2.contains

[]

In [5]:
drawer2.put_in(cup1)

True

In [6]:
drawer2.contains

## Ex.2: Features showcase

In [7]:
scene = Scene(objects=[drawer1, drawer2, cup1])
scene.info

Scene info. shape: [4, 4, 4]
drawer1, drawer, [0 0 0], closed, []
drawer2, drawer, [2 0 0], opened, ['cup1']
cup1, cup, [2 0 0], full
Robot: [3 0 3], closed, rotation: 0, attached: -


In [8]:
scene.collision_free()

True

In [10]:
scene.drawer2.pick_up(scene.cup1)

True

In [12]:
scene.info

Scene info. shape: [4, 4, 4]
drawer1, drawer, [0 0 0], closed, []
drawer2, drawer, [2 0 0], opened, []
cup1, cup, [2 0 0], full
Robot: [3 0 3], closed, rotation: 0, attached: -


Fake object move, normally **Move Actions** are used to move the object.

In [13]:
scene.cup1.position = scene.drawer1.position

In [14]:
scene.drawer1.open()

True

In [15]:
scene.drawer1.put_in(scene.cup1)

True

In [16]:
scene.info

Scene info. shape: [4, 4, 4]
drawer1, drawer, [0 0 0], opened, ['cup1']
drawer2, drawer, [2 0 0], opened, []
cup1, cup, [0 0 0], full
Robot: [3 0 3], closed, rotation: 0, attached: -


## Ex.3: Generate scene from dictionary
- Example has stacked objects -> `print_structure()` reveals the structure of an object

In [19]:
objects = {
    'drawer': {'position': np.array([3, 2, 0]),
        'type': 'drawer',
        'graspable': False,
        'pushable': False,
        'free': True,
        'size': 0.2,
        'opened': False,
        'above_str': 'cup1'},
    'cup1': {'position': np.array([3, 1, 0]),
        'type': 'cup',
        'graspable': True,
        'pushable': True,
        'free': True,
        'size': 0.01,
        'under_str': 'drawer',
        'above_str': 'cup2'},
    'cup2': {'position': np.array([1, 2, 0]),
        'type': 'cup',
        'graspable': True,
        'pushable': True,
        'free': True,
        'size': 0.01,
        'under_str': 'cup1'}
    }
scene = Scene(objects=objects, init='from_dict')

In [20]:
scene.cup1.print_structure()

Structure:
cup2
[cup1]
drawer


In [21]:
scene.cup2.print_structure()

Structure:
[cup2]
cup1
drawer


In [22]:
scene.drawer.print_structure()

Structure:
cup2
cup1
[drawer]


## Ex.3.1:

In [23]:
objects = {
    'drawer': {'position': np.array([3, 2, 0]),
        'type': 'drawer',
        'graspable': False,
        'pushable': False,
        'free': True,
        'size': 0.2,
        'opened': False,
        'contains_list': ['cup1', 'cup2'],
        'above_str': 'cup1'},
    'cup1': {'position': np.array([3, 1, 0]),
        'type': 'cup',
        'graspable': True,
        'pushable': True,
        'free': True,
        'size': 0.01,
        'under_str': 'drawer',
        'above_str': 'cup2'},
    'cup2': {'position': np.array([1, 2, 0]),
        'type': 'cup',
        'graspable': True,
        'pushable': True,
        'free': True,
        'size': 0.01,
        'under_str': 'cup1'}
    }
scene = Scene(objects=objects, init='from_dict')

In [24]:
scene.cup1.print_structure()

Structure:
cup2
[cup1]
drawer


In [25]:
scene.drawer.contains_list

['cup1', 'cup2']

In [26]:
scene.drawer.contains

In [27]:
scene.collision_free()

True

# Actions
- The layer which users interact with
- Each action has **Target Action** and **Target Object**, in future there might be **Target Measure**.

## Ex.1: Put cup into drawer

In [47]:
s = Scene(init='drawer_and_cup')
s.r.eef_position = s.drawer.position
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup, cup, [2 1 0], empty
Robot: [2 0 0], opened, rotation: 0, attached: -


In [48]:
print(Actions.do(s, ('pick_up', 'cup')))
s.info

('pick_up', 'cup') done!
True
Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup, cup, [2 1 0], empty
Robot: [2 0 0], opened, rotation: 0, attached: cup


In [49]:
Actions.do(s, ('put', 'drawer'))
s.info

Action cannot be performed!
Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup, cup, [2 1 0], empty
Robot: [2 0 0], opened, rotation: 0, attached: cup


## Ex.2: Pour cup one into cup two

In [10]:
s = Scene(init='drawer_and_cups', random=False)
s.cup1.fill()
s.cup1.info
s.cup2.info

cup1, cup, [2 1 0], full
cup2, cup, [2 2 0], empty


In [11]:
Actions.do(s, ('pick_up', 'cup1'))
Actions.do(s, ('pour', 'cup2'))
s.cup1.info
s.cup2.info

('pick_up', 'cup1') done!
('pour', 'cup2') done!
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], full


In [12]:
s.cup1.direction

array([0, 0, 1])

In [13]:
s.r.eef_direction

0

In [14]:
s.cup1.info
s.cup2.info

cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], full


In [16]:
s.cup2.rotate([0,1,0])
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: cup1


## Ex.3: Pour cup one into drawer

In [23]:
s = Scene(init='drawer_and_cups', random=False)
s.cup1.fill()
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], full
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [24]:
Actions.do(s, ('pick_up', 'cup1'))
Actions.do(s, ('pour', 'drawer'))
s.cup1.info

('pick_up', 'cup1') done!
('pour', 'drawer') done!
cup1, cup, [2 1 0], empty


In [25]:
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: cup1


## Ex.4: Spill the cup

In [34]:
s.cup2.rotate([1,0,0])
print(s.cup2)

cup2, cup, [2 2 0], empty


In [36]:
s = Scene(init='drawer_and_cups')
s.r.eef_position = s.cup1.position
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], opened, []
cup1, cup, [2 1 0], full
cup2, cup, [2 2 0], empty
Robot: [2 1 0], opened, rotation: 0, attached: -


In [37]:
Actions.do(s, ('push','cup1'))
s.cup1.position

('push', 'cup1') done!


array([3, 2, 0])